# Парсим онлайн-магазин c книгами на питоне: как обойти серверную блокировку.

[Books to scrape](https://books.toscrape.com/index.html)

# 1. Вламываемся в хранилище

## 1.1. Что мы хотим получить

Итак, мы хотим распарсить [books.toscrape.com](https://books.toscrape.com/index.html) и получить кучу разных переменных:


- **Name** – название книги
- **Rating** – оценка книги (1-5 звёзд)
- **Price** – цена книги
- **Availability** – наличие на складе
- **Category** – категория книги
- **UPC** – код книги
- **URL** – ссылка на книгу

После скачивания и чистки данных от мусора, можно будет заняться строительством моделей. Например, попытаться предсказать популярность книги по её параметрам. Но это все позже, а сейчас познакомимся с парой определений:

* **Парсер** — это скрипт, который грабит информацию с сайта
* **Краулер** — это часть парсера, которая **бродит по ссылкам**
* **Краулинг** — это переход по страницам и ссылкам
* **Скрапинг** — это **сбор данных** со страниц
* **Парсинг** — это сразу **и краулинг и скрапинг**!


## 1.2.  Что такое HTML

**HTML (HyperText Markup Language)**  — это такой же **язык разметки** как **Markdown** или **LaTeX**. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта.

Можно увидеть, что HTML-страница это ни что иное как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы
- `<tr>` – разделитель для строк в таблице
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное **дерево** с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега могут быть свои **потомки (дети)** — те теги, которые вложены в него, и свои родители.

Например, HTML-древо страницы может выглядеть вот так:

    <html>
    <head> Заголовок </head>
    <body>
        <div>
            Первый кусок текста со своими свойствами
        </div>
        <div>
            Второй кусок текста
                <b>
                    Третий, жирный кусок
                </b>
        </div>
        Четвёртый кусок текста        
    </body>
    </html>            
    
    


Можно работать с этим html как с текстом, а можно как с деревом. **Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!**

Вручную обходить эти деревья не очень приятно, поэтому есть специальные языки для обхода деревьев.



## 1.3. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль **`requests`**. Подгрузим его. За компанию подгрузим ещё парочку дельных пакетов.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для времени

Для наших благородных исследовательских целей **нужно собрать данные по каждой книге**  с соответствующей ей страницы. Но для начала **нужно получить адреса этих страниц** . Поэтому открываем основную страницу со всеми выложенными книгами.



Со страницы всех книг https://books.toscrape.com/index.html мы  будем тащить ссылки на каждую из перечисленных книг. Сохраним в переменную **`main_page_link`** адрес основной страницы и откроем её при помощи библиотеки **`requests`**.

In [3]:
main_page_link = 'https://books.toscrape.com/index.html'

Не будем показывать серверу, что мы сидим на питоне и используем библиотеку requests. Это может вызвать у него некоторые подозрения относительно наших благих намерений и он решит нас безжалостно отвергнуть.


Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации **фейкового юзер-агента**.


Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). **При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос**:

In [4]:
!pip install fake_useragent

In [5]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [6]:
UserAgent().chrome

'Mozilla/5.0 (iPhone; CPU iPhone OS 17_6_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/135.0.7049.53 Mobile/15E148 Safari/604.1'

In [ ]:
UserAgent().safari

'Mozilla/5.0 (iPhone; CPU iPhone OS 18_1_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.1.1 Mobile/15E148 Safari/604.1'

## 1.4. Извлечение HTML кода со страницы

In [7]:
# request meme page with chrome user agent
response = requests.get(main_page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [8]:
type(response)

requests.models.Response

Замечательно, наша небольшая маскировка сработала и обманутый сервер покорно выдал благословенный 200 ответ — соединение установлено и данные получены, всё чудесно! Посмотрим, что же все-таки мы получили.

In [9]:
html = response.content

In [10]:
html[:2000]

b'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" hre

In [11]:
html[-1000:]

b's"></script>\n            <script>window.jQuery || document.write(\'<script src="static/oscar/js/jquery/jquery-1.9.1.min.js"><\\/script>\')</script>\n        \n  \n\n\n        \n        \n    \n        \n    <!-- Twitter Bootstrap -->\n    <script type="text/javascript" src="static/oscar/js/bootstrap3/bootstrap.min.js"></script>\n    <!-- Oscar -->\n    <script src="static/oscar/js/oscar/ui.js" type="text/javascript" charset="utf-8"></script>\n\n    <script src="static/oscar/js/bootstrap-datetimepicker/bootstrap-datetimepicker.js" type="text/javascript" charset="utf-8"></script>\n    <script src="static/oscar/js/bootstrap-datetimepicker/locales/bootstrap-datetimepicker.all.js" type="text/javascript" charset="utf-8"></script>\n\n\n        \n        \n    \n\n    \n\n\n        \n        <script type="text/javascript">\n            $(function() {\n                \n    \n    \n    oscar.init();\n\n    oscar.search.init();\n\n            });\n        </script>\n\n        \n        <!-- V

In [12]:
len(html)

51294

In [13]:
type(html)

bytes

Выглядит непонятно и неудобно. как насчет сварить из этого дела что-то покрасивее? Например, красивый суп.

## 1.5. Парсинг HTML кода - Красивый суп



Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** (тут есть гиперссылка на лучшего друга человека — документацию) был назван в честь стишка про красивый суп из Алисы в стране чудес.

Красивый суп — это совершенно волшебная библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием **`BeautifulSoup`** — скорее всего, не то, что нам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Нужно будет установить пакет **`beautifulsoup4`**. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — **`bs4`**, а импортировать функцию под названием **`BeautifulSoup`**. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

**С необработанным XML кодом страницы пакет также работает** (XML — это исковерканый и превращённый в диалект, с помощью своих команд, HTML). Для того, чтобы пакет корректно работал с XML разметкой, придётся в довесок ко всему нашему арсеналу установить пакет `xml`.

In [14]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [15]:
soup = BeautifulSoup(html, 'html.parser') # В опции также можно указать lxml,
                                         # если предварительно установить одноименный пакет

In [16]:
type(soup)

bs4.BeautifulSoup

In [17]:
print(soup.prettify()[:2000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

Получим что-то вот такое:
    
```
<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHkUNWUU=","queueTime":0,"applicationTime":24,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=
```

Стало намного лучше, не правда ли? Что же лежит в переменной **`soup`**? Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно **бродить по HTML-дереву** страницы, **искать детей**, **родителей** и вытаскивать их!

Например, можно бродить по вершинам, указывая путь из тегов.

In [19]:
# show title tag
soup.html.head.title

<title>
    All products | Books to Scrape - Sandbox
</title>

In [18]:
type(soup.html.head.title)

bs4.element.Tag

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [20]:
# show title tag text which you usually see in the tab of your browser
soup.html.head.title.text

'\n    All products | Books to Scrape - Sandbox\n'

In [21]:
type(soup.html.head.title.text)

str

In [22]:
# delete leading and trailing newlines, tabs and spaces
soup.html.head.title.text.strip()

'All products | Books to Scrape - Sandbox'

In [23]:
type(soup.html.head.title.text.strip())

str

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега **`div`** и имеет класс **`side_categories`**.

In [24]:
# get element representing left sidebar menu
obj = soup.find('div', attrs = {'class':'side_categories'})
print(obj.prettify()[:500])

<div class="side_categories">
 <ul class="nav nav-list">
  <li>
   <a href="catalogue/category/books_1/index.html">
    Books
   </a>
   <ul>
    <li>
     <a href="catalogue/category/books/travel_2/index.html">
      Travel
     </a>
    </li>
    <li>
     <a href="catalogue/category/books/mystery_3/index.html">
      Mystery
     </a>
    </li>
    <li>
     <a href="catalogue/category/books/historical-fiction_4/index.html">
      Historical Fiction
     </a>
    </li>
    <li>
     <a href="


In [25]:
obj.get("class")

['side_categories']

**При работе с классами нужно быть внимательными!**

Давайте попробуем найти элемент, который лежит внутри тега **`header`** и имеет класс **`header`**.

In [26]:
obj_2 = soup.find('header', attrs = {'class':'header'})
obj_2.get("class")

['header', 'container-fluid']

In [27]:
print(obj_2.prettify()[:70])

<header class="header container-fluid">
 <div class="page_inner">
  <d


Вопреки нашим ожиданиям, вытащенный объект имеет класс **`"header container-fluid"`**.

`BeautifulSoup4` расценивает аттрибуты **`class`** как набор отдельных значений, поэтому **`"header container-fluid"`** для библиотеки равносильно **`["header", "container-fluid"]`**, а указанное нами значение этого класса **`"header"`** входит в этот список.


Чтобы избежать такой неприятной ситуации и проходов по ненужным нам ссылкам, придется воспользоваться собственной функцией и **задать строгое соответствие**:

In [28]:
'header' in ['header', 'container-fluid']

True

In [29]:
obj_2 = soup.find(lambda tag: tag.name == 'header' and tag.get('class') == ['header'])
obj_2

In [30]:
print(obj_2)

None


В коде HTML нет тега, который строго соответсвует нашему критерию. Поэтому, мы получаем объект `None`.

## Извлечение ссылок

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём! Вытащим ссылку на эту страницу. Сделать это можно по атрибуту `href`, в котором лежит наша ссылка.

In [31]:
# find first a-tag element on webpage
obj = soup.find('a')

In [32]:
obj

<a href="index.html">Books to Scrape</a>

In [33]:
# get url from this object
obj.attrs['href']

'index.html'

Обратите внимание, что после всех этих безумных преобразований у данных поменялся тип. Теперь они `str`. Это означет, что с ними можно работать как с текстом и пускать в ход для отсеивания лишней информации регулярные выражения.

In [34]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Тип данных до вытаскивания ссылки: <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки: <class 'str'>


Если несколько элементов на странице обладают указанным адресом, то метод **`find` вернёт только самый первый** .  Чтобы **найти все элементы** с таким адресом, нужно использовать метод **`findAll`**, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с книгами.

In [35]:
# get all tags
all_link_tags = soup.findAll(lambda tag: tag.name == 'a')
all_link_tags[:3]

[<a href="index.html">Books to Scrape</a>,
 <a href="index.html">Home</a>,
 <a href="catalogue/category/books_1/index.html">
                             
                                 Books
                             
                         </a>]

In [36]:
len(all_link_tags)

94

In [37]:
# clean text inside tags

cleaned_link_tags = []

for link in all_link_tags:
    if link.string:  # Check if the tag has a text node
        link.string = link.string.strip()  # Clean up the text inside the tag
    cleaned_link_tags.append(link)

In [38]:
all_link_tags[:10]

[<a href="index.html">Books to Scrape</a>,
 <a href="index.html">Home</a>,
 <a href="catalogue/category/books_1/index.html">Books</a>,
 <a href="catalogue/category/books/travel_2/index.html">Travel</a>,
 <a href="catalogue/category/books/mystery_3/index.html">Mystery</a>,
 <a href="catalogue/category/books/historical-fiction_4/index.html">Historical Fiction</a>,
 <a href="catalogue/category/books/sequential-art_5/index.html">Sequential Art</a>,
 <a href="catalogue/category/books/classics_6/index.html">Classics</a>,
 <a href="catalogue/category/books/philosophy_7/index.html">Philosophy</a>,
 <a href="catalogue/category/books/romance_8/index.html">Romance</a>]

In [39]:
# observe number of found link tags
len(all_link_tags)

94

Осталось очистить полученный список от мусора (синтаксиса тегов), чтобы получить только ссылки:

In [40]:
# get urls from tags
page_links = [link.attrs['href'] for link in all_link_tags]

In [41]:
page_links

['index.html',
 'index.html',
 'catalogue/category/books_1/index.html',
 'catalogue/category/books/travel_2/index.html',
 'catalogue/category/books/mystery_3/index.html',
 'catalogue/category/books/historical-fiction_4/index.html',
 'catalogue/category/books/sequential-art_5/index.html',
 'catalogue/category/books/classics_6/index.html',
 'catalogue/category/books/philosophy_7/index.html',
 'catalogue/category/books/romance_8/index.html',
 'catalogue/category/books/womens-fiction_9/index.html',
 'catalogue/category/books/fiction_10/index.html',
 'catalogue/category/books/childrens_11/index.html',
 'catalogue/category/books/religion_12/index.html',
 'catalogue/category/books/nonfiction_13/index.html',
 'catalogue/category/books/music_14/index.html',
 'catalogue/category/books/default_15/index.html',
 'catalogue/category/books/science-fiction_16/index.html',
 'catalogue/category/books/sports-and-games_17/index.html',
 'catalogue/category/books/add-a-comment_18/index.html',
 'catalogue/ca

Мы получили **все ссылки с главной страницы**. Теперь мы х**отим собрать ссылки на каждую книгу**. На этом сайте это можно сделать двумя сособами:

1. **Через общий раздел `Books`** где сохранены все книги. Тут нам нужно будет извлекать **ссылку на следующую страницу**, т.к. за раз на странице отображены 20 книг.
2. **Через разделы отдельных категорий книг**. Там нам тоже иногда придётся "листать".
  * **Преимущество**: мы **сразу знаем категорию** всех книг на такой странице и можем напрямую записать её в нашу таблицу.

Так как нам в любом случае придётся "листать", давайте **ВЫБЕРЕМ ПЕРВЫЙ СПОСОБ**. Информацию о категории мы найдём и на каждой странице отдельных книг.



**Нам нужно будет каким-то образом забраться на следующую страницу**.

На сайте это можно делать просто щёлкая на кнопку **`next`** в правом нижнем углу, если она есть. javascript-функции откроют новую страницу с книгами. Но сейчас трогать эти функции не хочется.

Обычно, все параметры, которые мы устанавливаем на сайте для поиска, отображаются на структуре хрефа. Книги не являются исключением. Если мы хотим получить первую порцию книг, мы должны будем обратиться к сайту по ссылке

                `https://books.toscrape.com/index.html`


Если мы захотим получить вторую поцию с двадцатью книгами, нам придётся немного видоизменить ссылку. Если нажать на кнопку **`next`**, то мы получим эту ссылку:


            `https://books.toscrape.com/catalogue/page-2.html`

Если мы присмотримся внимательно к последнему элементу в нашем списке **`page_links`**, то увидим, что это и есть ссылка на следующую страницу. Можно брать следующую страницу либо из списка всех ссылок на текущей странице, либо через поиск html-тэга с кнопкой **`next`**, извлекая оттуда ссылку или через простую самостоятельную генерацтю ссылок нужного формата.

Третью страницу мы получим через:

            `https://books.toscrape.com/catalogue/page-3.html`

И если попробовать открыть страницу, заканчивающуюся на **`page-1.html`**, то мы увидим, что она тоже существует и полностью соответствует стартовой странице.

Значит, мы просто можем подставлять всё больший и больший индекс страницы, пока не заметим, что следующей страницы не существует. Таким незатейливым образом мы сможем пройтись по всем страницам и ограбить книжный магазин.

Наконец, обернем в красивую функцию все-все манипуляции, проделанные выше:

In [42]:
from typing import List

def getBookPageLinks(page_number: int) -> List[str]:
    """
        Возвращает список ссылок на книги, полученный с текущей страницы

        page_number: int
            номер страницы для парсинга

    """
    # составляем ссылку на страницу поиска
    page_link = f'https://books.toscrape.com/catalogue/page-{page_number}.html'

    # запрашиваем данные по текущей странице
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])

    # ищем все div tags с class 'image_container'
    book_containers = soup.findAll(lambda tag: tag.name == 'div' and tag.get('class') == ['image_container'])

    # извлекаем href из ПЕРВОГО <a> tag внутри каждого div
    book_links = [container.find('a')['href'] for container in book_containers if container.find('a')]

    return book_links

Протестируем функцию и убедимся, что всё хорошо

In [43]:
book_links_page_3 = getBookPageLinks(3)
len(book_links_page_3)
book_links_page_3[0]

'slow-states-of-collapse-poems_960/index.html'

In [44]:
# check if URL exists
try:
    response = requests.get(book_links_page_3[0], headers={'User-Agent': UserAgent().chrome})
    print("Success!")
except:
    print("Page does not exist!")

Page does not exist!


Мы видим, что мы получили **сокращённые ссылки**. Давайте **добавим** к каждой ссылке **префикс** сайта:

In [45]:
def add_page_prefix(prefix: str, list_of_urls: List[str]) -> List[str]:
    """
        Добавляет к каждой ссылке на книгу префикс сайта

        prefix: str
            префикс для действующей ссылки

        list_of_urls: List[str]
            список кратких ссылок без префиксов
    """
    extended_urls = []

    for url in list_of_urls:
        extended_urls.append(prefix + url)
    return extended_urls

In [46]:
book_prefix = 'https://books.toscrape.com/catalogue/'

book_links_page_3_extended = add_page_prefix(prefix=book_prefix, list_of_urls=book_links_page_3)

In [47]:
for page_url in book_links_page_3_extended:
  print(page_url)

https://books.toscrape.com/catalogue/slow-states-of-collapse-poems_960/index.html
https://books.toscrape.com/catalogue/reasons-to-stay-alive_959/index.html
https://books.toscrape.com/catalogue/private-paris-private-10_958/index.html
https://books.toscrape.com/catalogue/higherselfie-wake-up-your-life-free-your-soul-find-your-tribe_957/index.html
https://books.toscrape.com/catalogue/without-borders-wanderlove-1_956/index.html
https://books.toscrape.com/catalogue/when-we-collided_955/index.html
https://books.toscrape.com/catalogue/we-love-you-charlie-freeman_954/index.html
https://books.toscrape.com/catalogue/untitled-collection-sabbath-poems-2014_953/index.html
https://books.toscrape.com/catalogue/unseen-city-the-majesty-of-pigeons-the-discreet-charm-of-snails-other-wonders-of-the-urban-wilderness_952/index.html
https://books.toscrape.com/catalogue/unicorn-tracks_951/index.html
https://books.toscrape.com/catalogue/unbound-how-eight-technologies-made-us-human-transformed-society-and-broug

In [48]:
# check if extended URL exists
try:
    response = requests.get(book_links_page_3_extended[0], headers={'User-Agent': UserAgent().chrome})
    print("Success!")
except:
    print("Page does not exist!")

Success!


Отлично, функция работает и теперь мы теоретически можем достать ссылки на все книги, для чего нам придется пройтись по 50 страницам. Прежде чем расстраивать сервер таким количеством запросов, посмотрим, как доставать всю необходимую информацию о конкретной книге.

Давайте теперь соберём ссылки на все книги со всех страниц!

In [49]:
# get urls of all books in store
book_links = []

book_prefix = 'https://books.toscrape.com/catalogue/'

page_number = 1
num_subsequent_errors = 0

while True:
    if page_number % 5 == 0:
        print(f'processing page {page_number} ...')
    # Attempt to get book links from the current page
    book_links_from_current_page = getBookPageLinks(page_number=page_number)
    if book_links_from_current_page != []:
        # add prefix
        book_links_from_current_page = add_page_prefix(prefix=book_prefix, list_of_urls=book_links_from_current_page)
        book_links.extend(book_links_from_current_page)
        # Reset error counter since this page was successfully read
        num_subsequent_errors = 0
        # Increment to the next page
        page_number += 1
    else:
        print(f'Page {page_number} could not be read or it does not exist!')
        # Increment error counter
        num_subsequent_errors += 1
        page_number += 1  # Still increment page number to move forward

        # Stop if 3 errors in a row occur
        if num_subsequent_errors == 3:
            print("Stopping due to 3 consecutive errors.")
            break

processing page 5 ...
processing page 10 ...
processing page 15 ...
processing page 20 ...
processing page 25 ...
processing page 30 ...
processing page 35 ...
processing page 40 ...
processing page 45 ...
processing page 50 ...
Page 51 could not be read or it does not exist!
Page 52 could not be read or it does not exist!
Page 53 could not be read or it does not exist!
Stopping due to 3 consecutive errors.


In [ ]:
print(f'{len(book_links)} book urls could be found in the book store.')

1000 book urls could be found in the book store.


## 1.6 Финальная подготовка к грабежу

По аналогии со ссылками можно вытащить что угодно. Для этого надо сделать несколько шагов:

1. Открываем страничку с книгой
2. Находим любым способом тег для нужной нам информации
3. Пихаем всё это в красивый суп
4. ......
5. Profit

Для закрепления информации в голове любознательного читателя, **вытащим всю интересующую нас информацию о книге**.

А в качестве примера возьмем самую первую книгу на сайте.

Как и прежде, для начала сохраним ссылку на страницу в переменную и вытащим по ней контент.

Давайте вытащим **название книги** (Name), её **рейтинг** (Rating), **стоимость** (Price), **наличие** (Availability), **жанр** (Category), **код книги** (UPC) и **ссылку** на саму книгу (URL).

In [50]:
def getStats(soup) -> dict:
    """

    """
    # get main part with main info (we assume it is always given)
    product_main = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['col-sm-6', 'product_main'])

    #####################################
    #  get BOOK NAME from main section  #
    #####################################
    if product_main:
      name = product_main.find('h1').text.strip()
      if not name:
        name = 'incognito'
    else:
      name = 'incognito'

    ################
    #  get RATING  #
    ################
    # get tag holding RATING stars
    rating_element = product_main.find(lambda tag: tag.name == 'p' and 'star-rating' in tag.get('class'))
    # get rating as word
    if rating_element:
        rating_word = rating_element.get('class')[-1]
        if rating_word:
            # map rating word to a number
            if rating_word == "One":
                rating = 1
            elif rating_word == "Two":
                rating = 2
            elif rating_word == "Three":
                rating = 3
            elif rating_word == "Four":
                rating = 4
            elif rating_word == "Five":
                rating = 5
    else:
        rating = -1

    ###############
    #  get PRICE  #
    ###############
    price = product_main.find(lambda tag: tag.name == 'p' and tag.get('class') == ['price_color']).text.strip()
    if not price:
        price = '£00.00'

    ##########################################
    #  get AVAILABILITY (always 'in stock')  #
    ##########################################
    availability = 'in stock' # TODO update if new values arise

    ##################
    #  get CATEGORY  #
    ##################
    category_element = soup.find(lambda tag: tag.name == 'ul' and tag.get('class') == ['breadcrumb'])
    if category_element:
        # Get the third <li> element (index 2)
        third_li = category_element.find_all('li')[2]
        # Extract the text from the <a> tag inside the third <li>
        if third_li.find('a'):
            category = third_li.find('a').text.strip()
    else:
        category = "Unknown"

    ##################
    #  get UPC CODE  #
    ##################
    # Find the <table> with the class 'table table-striped'
    table = soup.find('table', class_='table table-striped')

    if table:
        # Find the row where <th> contains "UPC" and get the corresponding <td> value
        for row in table.find_all('tr'):
            th = row.find('th')
            if th and th.get_text(strip=True) == 'UPC':
                upc = row.find('td').get_text(strip=True)
                break
    else:
        upc = "Unknown"

    # collect and return results
    return {
        "name": name,
        "rating": rating,
        "price": price,
        "availability": availability,
        "category": category,
        "upc": upc,
    }


На самом деле, **парсеры — дело непредсказуемое. Часто страницы, которые мы парсим, имеют очень неоднородну структуру**. Например, если мы парсим книги, на части страниц может быть указано описание, а на части нет. Как только код впервые натыкается на отсутствие описания, он выдаёт ошибку и останавливается. Чтобы нормально собрать все данные, **приходится прописывать исключения**. Вроде бы, хранилище книг хорошо оборудовано и никаких внештатных ситуаций происходить не должно. Тем не менее, очень не хочется проснуться утром и увидеть, что код сделал 20 итераций, нарвался на ошибку и отрубился.  Чтобы такого не произошло, можно, например, использовать конструкцию `try - except` и просто обрабатывать неугодные нам ошибки. Про исключения можно почитать [на просторах интернета](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html). В нашем же случае до ошибки можно и не доводить, а предварительно проверять, есть ли необходимый элемент на странице или нет при помощи обычного `if - else`, и уже после этого пытаться его распарсить.



## Truthy & Falsy python
[Подробнее](https://www.freecodecamp.org/news/truthy-and-falsy-values-in-python/)

Такой код позволяет обезопасить себя от ошибок в коде. В данном случае, мы можем переписать всю конструкцию с `if - else` в виде одной удобной строки. Эта строка проверит полон ли респонса `meme_status` и ежели нет, то выдаст пустоту.

In [ ]:
"Hello" if True else "Bye"

'Hello'

In [ ]:
1 if ("a" in ['a', 'b']) or ('c' in [1, 4]) else 0

1

In [ ]:
1 if ("a" in ['a', 'b']) and ('c' in [1, 4]) else 0

0

Наконец, создадим функцию, возвращающую всю информацию по текущей книги

In [ ]:
def getBookData(book_page: str) -> dict:
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными

        book_page: string
            ссылка на страницу с книгой

    """

    # запрашиваем данные по ссылке
    response = requests.get(book_page, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки
        return response.status_code

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанную функцию парсим информацию
    data_row = getStats(soup)

    # добавляем ссылку
    data_row['book_url'] = book_page

    return data_row

In [ ]:
data_row = getBookData(book_page='https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html')

In [ ]:
data_row

{'name': 'A Light in the Attic',
 'rating': 3,
 'price': '£51.77',
 'availability': 'in stock',
 'category': 'Poetry',
 'upc': 'a897fe39b1053632',
 'book_url': 'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'}

А теперь подготовим табличку, чтобы в неё записывать всё ~~награбленные~~ честно полученные данные, добавим в неё первую полученную строку и полюбуемся на результат

In [ ]:
final_df = pd.DataFrame(columns=['Name',  'Rating', 'Price', 'Availability',
                                 'Category', 'UPC', 'BOOK_URL'])

In [ ]:
# final_df = final_df.append(data_row, ignore_index=True).dropna(axis = 1) # NOTE deprecated in pandas 2.x

# Assuming final_df and data_row are already defined
final_df = pd.concat([final_df, pd.DataFrame([data_row])], ignore_index=True).dropna(axis=1)

In [ ]:
final_df

,name,rating,price,availability,category,upc,book_url
0,A Light in the Attic,3.0,£51.77,in stock,Poetry,a897fe39b1053632,https://books.toscrape.com/catalogue/a-light-i...


Первая книга оказалась в наших рукак. Еще раз убедимся что всё работает — пройдемся по списку из ссылок на все 1000 книги, полученных ранее в перменной **`book_links`**.

In [ ]:
# from tqdm import tqdm_notebook
from tqdm import tqdm, tqdm_notebook

In [ ]:
# NOTE: this cell takes long! (ca. minutes)

counter = 1

for book_link in tqdm_notebook(book_links):
    try:
        # Get book data
        data_row = getBookData(book_link)

        # Convert data_row (a dict) to a DataFrame
        if isinstance(data_row, dict):
            data_row = pd.DataFrame([data_row])

        # Concatenate the new data row to final_df
        final_df = pd.concat([final_df, data_row], ignore_index=True).dropna(axis=1)

        # Sleep to prevent overwhelming the server
        time.sleep(0.3)
        if counter % 100 == 0:
            print(f"OK after {counter} book pages")
        counter += 1
    except Exception as e:
        print(f"Exception occurred with {book_link}: {e}")
        counter += 1
        continue

In [ ]:
final_df = final_df.drop_duplicates().dropna(axis = 1)

In [ ]:
# show number of rows and columns
final_df.shape

(349, 7)

In [ ]:
final_df.head()

,name,rating,price,availability,category,upc,book_url
0,A Light in the Attic,3.0,£51.77,in stock,Poetry,a897fe39b1053632,https://books.toscrape.com/catalogue/a-light-i...
2,Tipping the Velvet,1.0,£53.74,in stock,Historical Fiction,90fa61229261140a,https://books.toscrape.com/catalogue/tipping-t...
3,Soumission,1.0,£50.10,in stock,Fiction,6957f44c3847a760,https://books.toscrape.com/catalogue/soumissio...
4,Sharp Objects,4.0,£47.82,in stock,Mystery,e00eb4fd7b871a48,https://books.toscrape.com/catalogue/sharp-obj...
5,Sapiens: A Brief History of Humankind,5.0,£54.23,in stock,History,4165285e1663650f,https://books.toscrape.com/catalogue/sapiens-a...


In [ ]:
final_df.tail()

,name,rating,price,availability,category,upc,book_url
345,A Study in Scarlet (Sherlock Holmes #1),2.0,£16.73,in stock,Mystery,63ee5bc46066a8a8,https://books.toscrape.com/catalogue/a-study-i...
346,A Series of Catastrophes and Miracles: A True ...,2.0,£56.48,in stock,Add a comment,c1379d3744f1dd2c,https://books.toscrape.com/catalogue/a-series-...
347,A People's History of the United States,2.0,£40.79,in stock,Add a comment,1ad06aed9349af46,https://books.toscrape.com/catalogue/a-peoples...
348,A Man Called Ove,1.0,£39.72,in stock,Fiction,8682c03de6a844ff,https://books.toscrape.com/catalogue/a-man-cal...
349,A Distant Mirror: The Calamitous 14th Century,3.0,£14.58,in stock,History,d5e0526e1ab682a3,https://books.toscrape.com/catalogue/a-distant...


Отлично! Всё работает, книги качаются, данные наполняются и всё было бы хорошо, если бы не одно но — количество запросов, которое нам придётся сделать, чтобы всё получить.

## 2.2 Тор - сын Одина

Иногда **серверу надоедает общаться с одним и тем же человеком, делающим кучу запросов и сервер банит его**. К сожалению, не только у людей запас терпения ограничен.

Приходится маскироваться. Для такой маскировки можно использовать разные способы, более того, один из них мы уже использовали, когда притворились человеком в нашем `request-header`. Для текущей же задачи, **когда нас вероломно заблокировали по IP, нужно искать способы помощнее, чтобы иметь возможность этот IP менять**. Конечно, как вариант можно было бы использовать прокси-сервера, тогда мы бы имели в запасе некоторое количество разных IP адресов, которые можно подставлять по мере "забанивания". Однако в этом подходе есть пара проблем: первая - нужно где-то раздобыть эти прокси, вторая - а что если ограниченного числа адресов нам не хватит и нужно больше?

В таком случае лучше всего нам подойдёт [Tor](https://www.torproject.org/). Вопреки пропагандируемому мнению, Tor используется не только преступниками, педофилами и прочими нехорошими террористами. Это, мягко говоря, далеко не так, и мы, мирные собиратели данных, являемся тому подтверждением. Всем прелестям, связанным с работой Tor, можно было бы посвятить несколько больших статеек, что собственно говоря уже и сделано. Подробнее про это можно почитатать по следующим ссылкам:

* [Как работает Tor](https://geektimes.ru/post/277578/)
* [Методы анонимности в сети](https://habrahabr.ru/post/204266/)
* [Прокси-сервер с помощью Tor](https://habrahabr.ru/company/etagi/blog/315002/)



In [ ]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

In [ ]:
checkIP()

Current IP Address: 34.125.195.24





## Почиташки

* [Неплохая инструкция](https://jarroba.com/anonymous-scraping-by-tor-network/) о самостоятельном парсинге через Tor без использования чужих готовых классов.
* [Оригинальный кодекс](http://starwars.wikia.com/wiki/Code_of_the_Sith) адепта тёмной стороны силы.